# wearablehrv - documentation 

### A Python package designed for data preparation, pre-processing, feature exctraction, comparison, visualization, and statistical individual/group analysis of heart rate and heart rate variability outcome variables recorded from wearable devices that transmit raw interbeat intervals and timestamps. 
~ Amin Sinichi | Vrije Universiteit Amsterdam | 2023

**It is highly recommended that this pipeline is run via Jupyter Notebook, installed through Anaconda Navigator.** 

In [ ]:
pip install pingouin 
pip install hrv-analysis 
#pip install wearablehrv
pip install avro-python3

# -------------------------------------- Individual Pipeline --------------------------------------

## Introduction
This pipeline is used for each and every participant. The pre-processed data then is saved as a .csv file for group analysis. 

### Shape of datasets

Datasets should be saved as .csv file with this format:

* [Participant ID]_[name of the device].csv
* Example: P01_polar.csv

This dataset for each device is a continuous recording that contains all conditions. It should follow this structure: the first column is time in UNIX format, in milliseconds; the second column contains inter-beat intervals. This can, for example, be recorded via the HRV Logger application developed by Marco Altini (https://www.hrv.tools/hrv-logger-faq.html). Of note, the code then converts the timestamps to hh:mm:ss format. 

The following section needs to be modified based on your datasets:

* `pp`: the participant ID. Please ensure that this is exactly the same as the one used to save your files.
* `path`: the location where all your files for a given participant are located. Copy and paste the path by removing the asterisks and replacing the text between them.
* `conditions`: the experimental conditions you used should be listed, for example: `['sitting', 'breathing']`.
* `devices`: the devices you used should be listed, for example: `['empatica', 'polar']`. Please ensure that this is exactly the same as the names used to save your files. Additionally, ensure that you always specify the criterion device as the last element of the list.
* `criterion`: specify the name of your criterion device, for example: `polar`. Please ensure that this is exactly the same as the name used to save your files.

***Side tip1:*** If you are by any chance using the VU-AMS device as your criterion (https://vu-ams.nl/), the inter-beat interval saved as a .txt file can be left as it is, and there is no need to modify it further. Just ensure that you name the device as "vu" in your files and also set the criterion as `vu`.

In [ ]:
# importing module
import wearablehrv

print ("Imported Successfully!")

In [ ]:
pp = "test" # Define the participant ID here
path = r'***COPY-YOUR-ADDRESS-HERE***/'.replace('\\', '/') # Define where the files are located 
path = r'C:\Users\msi401\surfdrive\Wearable Validation\Raw Data\Participants/'.replace('\\', '/') # Define where the files are located 
path = r'C:\Users\msi401\OneDrive - Vrije Universiteit Amsterdam\PhD\Data\Coding\Validation Study\wearable-hrv\tests\test_individual/'.replace('\\', '/') # Define where the files are located 
conditions = ['sitting', 'arithmetic', 'recovery', 'standing', 'breathing', 'neurotask', 'walking', 'biking']
devices = ["kyto", "heartmath", "rhythm", "empatica", "vu"]  ### MAKE SURE TO PUT THE CRITERION DEVICE THE LAST ONE IN THE LIST #####
criterion = "vu"

print ("All Imported Successfully!")

---
#### Skippable if You Are Not Using Empatica and/or LabFront


***Side tip2:*** <u>LabFront Integration</u>. By using the `labfront_conversion` function, you can convert the data file that was exported from the LabFront platform (https://www.labfront.com/) into a format that is readble for this pipeline. Save the file in the same path. Use the `file_name` to indicate the name of the file, and be sure to add ".csv" at the end. Here is an example file name:

- 000001_garmin-device-bbi_P01_162a8cdc_garmin-device_5d1cdbe5.csv

Make sure that `device_name` matches the name you used in the `devices` variable. For instance, if you are recording data from a Garmin watch via LabFront, you could save it as "garmin" in both `devices` and `device_name` variables. Enter the date for which you want to do the validation (e.g., '2023-04-04'). 


***Side tip3:*** <u>Empatica Integration</u>. If you are using Empatica Embrace Plus, ensure that you place the entire folder (e.g., using CyberDuck) received for each participant within a day, which includes "digital_biomarkers" and "raw_data" folders. To organize this folder, name it as [participantID]_empatica (e.g., P00_empatica), and place it in the path. Next, run the `empatica_conversion` function to perform the transformation automatically and generate a .csv file that is compatible with the pipeline.

In [ ]:
# skip this if you are not using LabFront or if your files are already converted into a proper format for this pipeline

file_name = ""
device_name = ""
date = ""

wearablehrv.individual.labfront_conversion (path, pp, file_name, device_name, date)

In [ ]:
# skip this if you are not using Empatica or if your files are already converted into a proper format for this pipeline

wearablehrv.individual.empatica_conversion (path, pp)

---
## Defining events, importing data, and chopping data based on the events

"In order to define the events, you have two options. Firstly, you can already have a saved .csv file named as [Participant ID]_events.csv in your path with a shape similar to this:

| timestamp | conditions | datapoint |
|-----------|------------|-----------|
| 11:48:28  | sitting    | start     |
| 11:50:28  | sitting    | end       |
| 11:53:00  | breathing  | start     |
| 11:55:00  | breathing  | end       |

In this case, set `already_saved = True`. Secondly, you may not have such an events file. In this case, set `already_saved = False`. By running the code, a graphical user interface will pop up, where you can define your events.
If you set `save_as_csv = True`, this event file will be saved in your path location. 

**WARNING:** Be careful not to overwrite your current event file.


In [ ]:
events = wearablehrv.individual.define_events (path, pp, conditions, already_saved= True, save_as_csv= False)

`Data` refers to the imported continuous data for all of your devices, while `data_chopped` no longer represents continuous data. Instead, it chops each dataset for each device based on specific events (conditions).

In [ ]:
data = wearablehrv.individual.import_data (path, pp, devices)
data_chopped = wearablehrv.individual.chop_data (data, conditions, events, devices)

## Visual Inspection: align and modify the length of the recordings 

By calling the `visual_inspection` function, a graphical user interface (GUI) will be displayed, allowing you to plot and navigate through each device and condition against your criterion device. This enhanced interface offers the following capabilities:

1. **Inspect Data**: Visually inspect and compare your data.
2. **Lag Alignment**: Adjust the time lag between your device's data and the criterion device. You can modify the lag in both seconds and milliseconds, thanks to the new precision dropdown.
3. **Trimming**: Define the start and end points of your data for both your selected device and the criterion.
4. **Full Lag Correction**: It lets you decide whether the lag adjustment for a specific device should be applied across all conditions or just the currently selected condition.

A recommended approach is to select a condition, inspect it across all devices, align the lags as necessary, and then move on to the next condition. Repeat this process until all conditions have been reviewed. After finishing the initial pass, it's a good practice to revisit and verify the alignments once more.

The `nibi_before_cropping` and `nibi_after_cropping` variables are optional and can be calculated to track the number of interbeat intervals detected in each condition for each device, both before and after aligning and trimming the data.

In [ ]:
nibi_before_cropping = wearablehrv.individual.calculate_ibi (data_chopped, devices, conditions)

In [ ]:
nibi_before_cropping['rhythm']['biking']

In [ ]:
wearablehrv.individual.visual_inspection (data_chopped, devices, conditions,criterion)

In [ ]:
nibi_after_cropping = wearablehrv.individual.calculate_ibi (data_chopped, devices, conditions)

### CheckPoint

Up to this point, we have made many time-consuming modifications to our dataset. Therefore, it is advisable to create a backup so that you can retrieve the version of the `data_chopped` up to this point. The `save_backup` function saves the `data_chopped` variable in `.pkl` format at your specified path location. In order to retrieve this file, you can call the `import_backup` function.

In [ ]:
wearablehrv.individual.save_backup (pp, path, data_chopped) #to save the file 

In [ ]:
data_chopped = wearablehrv.individual.import_backup (pp, path) #to read the file again

## Pre-processing: removing ectopic beats and movement artefact, and interpolating removed values

We rely on the hrvanalysis python module for pre-processing. You can read the documentation here: https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html. 

**Please Note**: It is important to note that the shape of the `data_chopped` variable changes and is converted to "rr intervals" only after executing this code. The reason for this change is that all subsequent analyses and steps will be performed solely on rr intervals. 

In [ ]:
data_pp, data_chopped = wearablehrv.individual.pre_processing (data_chopped, devices, conditions, method="karlsson", custom_removing_rule = 0.25, low_rri=300, high_rri=2000)

#### How to Skip Data Pre-processing and Still Run Your Code
If you wish to skip the data pre-processing step but still want to be able to execute the remaining code, please ensure that you run the following code:

In [ ]:
# skip this if you have already pre-processed your data

data_chopped = {device: {condition: list(data_chopped[device][condition]['rr']) for condition in conditions} for device in devices}
data_pp = data_chopped.copy()

## Plotting pre-processed vs. original data

We rely on the hrvanalysis python module for time domain and frequency domain feature extraction. You can read the documentation here: https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html. Before doing so, we can optionally save the number of detected artefact for each device and condition.

You can plot the preprocessed versus original data by running the `ibi_comparison_plot` function. The down row always shows your criterion device, and the up row shows the selected device.

A suggested approach is that you can experiment with the `pre_processing` function until you find the parameters that minimize the amount of artifact in your criterion device and optimize the amount of artifact in your other devices. To do so, make sure to first retrive your `data_chopped` variable. 

In [ ]:
artefact = wearablehrv.individual.calculate_artefact (data_chopped, data_pp, devices, conditions)

In [ ]:
wearablehrv.individual.ibi_comparison_plot (data_chopped, data_pp, devices, conditions, criterion, width=20 , height=10)

## Data analysis 

In [ ]:
time_domain_features, frequency_domain_features = wearablehrv.individual.data_analysis (data_pp, devices, conditions)

**Time Domain Features:**

- `mean_nni`: Mean of RR-intervals.
- `sdnn`: Standard deviation of the time interval between successive normal heart beats (i.e. the RR-intervals).
- `sdsd`: Standard deviation of differences between adjacent RR-intervals.
- `rmssd`: Square root of the mean of the sum of the squares of differences between adjacent NN-intervals. Reflects high frequency (fast or parasympathetic) influences on hrV (i.e., those influencing larger changes from one beat to the next).
- `median_nni`: Median Absolute values of the successive differences between the RR-intervals.
- `nni_50`: Number of interval differences of successive RR-intervals greater than 50 ms.
- `pnni_50`: Proportion derived by dividing nni_50 (The number of interval differences of successive RR-intervals greater than 50 ms) by the total number of RR-intervals.
- `nni_20`: Number of interval differences of successive RR-intervals greater than 20 ms.
- `pnni_20`: Proportion derived by dividing nni_20 (The number of interval differences of successive RR-intervals greater than 20 ms) by the total number of RR-intervals.
- `range_nni`: Difference between the maximum and minimum nn_interval.
- `cvsd`: Coefficient of variation of successive differences equal to the rmssd divided by mean_nni.
- `cvnni`: Coefficient of variation equal to the ratio of sdnn divided by mean_nni.
- `mean_hr`: Mean Heart Rate.
- `max_hr`: Maximum Heart Rate.
- `min_hr`: Minimum Heart Rate.
- `std_hr`: Standard deviation of heart rate.

**Frequency Domain Features:**

- `total_power`: Total power density spectral.
- `vlf`: Variance ( = power ) in HRV in the Very low Frequency (.003 to .04 Hz by default). Reflects an intrinsic rhythm produced by the heart which is modulated primarily by sympathetic activity.
- `lf`: Variance ( = power ) in HRV in the low Frequency (.04 to .15 Hz). Reflects a mixture of sympathetic and parasympathetic activity, but in long-term recordings, it reflects sympathetic activity and can be reduced by the beta-adrenergic antagonist propanolol.
- `hf`: Variance ( = power ) in HRV in the High Frequency (.15 to .40 Hz by default). Reflects fast changes in beat-to-beat variability due to parasympathetic (vagal) activity. Sometimes called the respiratory band because it corresponds to HRV changes related to the respiratory cycle and can be increased by slow, deep breathing (about 6 or 7 breaths per minute) and decreased by anticholinergic drugs or vagal blockade.
- `lf_hf_ratio`: lf/hf ratio is sometimes used by some investigators as a quantitative mirror of the sympatho/vagal balance.
- `lfnu`: Normalized lf power.
- `hfnu`: Normalized hf power.


## Plotting analysis results

`results_comparison_plot` creates comparison bar charts for time and frequency domain measures of HRV for each device, both for the original data and after pre-processing. This also enables you to fine-tune your pre-processing parameters. To do so, make sure to first retrive your `data_chopped` variable.

The `unfolding_plot` function helps visualize how the RMSSD (above plot) and heart rate (below plot) unfold over time in a given condition. It may be specifically interesting for the criterion device. By changing the `Sec` value, you can modify the time window within which the values are calculated in seconds.

In [ ]:
wearablehrv.individual.result_comparison_plot (data_chopped, time_domain_features, frequency_domain_features, devices, conditions, bar_width = 0.20, width = 20, height = 25)

In [ ]:
wearablehrv.individual.unfolding_plot (data_pp, devices, conditions, width = 10, height = 10)

In [ ]:
wearablehrv.individual.bar_plot (time_domain_features, frequency_domain_features, devices, conditions, width=20, height=25, bar_width = 0.20)

In [ ]:
wearablehrv.individual.line_plot (time_domain_features, frequency_domain_features, devices, conditions, width=20, height=25)

In [ ]:
wearablehrv.individual.radar_plot (time_domain_features, criterion, devices, conditions)

## Display the absolut values and changes between conditions 

Using the following code, you can view the absolute values of each condition for every device, along with the changes from one condition to another within a device. This makes it easier to analyze the `line_plot` function simultaneously.

In [ ]:
wearablehrv.individual.display_changes (time_domain_features, frequency_domain_features, devices, conditions)

## Saving Data

The parameters `artefact`, `nibi_before_cropping`, and `nibi_after_cropping` are optional. If you have calculated them, you can set them as inputs (e.g., `artefact = artefact`). Otherwise, set these parameters to `None`, and `N/D` will be replaced in that column. To save your data, set `save_as_csv = True`. The final table will then be saved in your path (you can redefine this) with the name [participant ID].csv.

In [ ]:
df = wearablehrv.individual.save_data(pp, path, time_domain_features, frequency_domain_features, data_pp, devices, conditions, events, artefact=None, nibi_before_cropping=None, nibi_after_cropping=None, save_as_csv=False)
df

# -------------------------------------- Group Pipeline --------------------------------------

## Introduction

Once you have run the previous sections for all your participants and saved the data as P00.csv files, you can then run the following functions for group analyses and visualization.

### Shape of datasets

Your data needs to be located in a folder. We propose that you save your data with the participant's ID, like this:

- P01.csv
- P02.csv
- etc.

Each file should contain a table similar to what you created with the individual script for each participant. Do not place any other files in this folder. 

In [1]:
# importing modules 

import wearablehrv

print ("Imported Successfully!")

Imported Successfully!


The following section needs to be modified based on your datasets:

* `path`: the location where all your files are located. Copy and paste the path by removing the asterisks and replacing the text between them.
* `conditions`: the experimental conditions you used should be listed, for example: `['sitting', 'breathing']`.
* `devices`: the devices you used should be listed, for example: `['polar', 'empatica']`. Please ensure that this is exactly the same as the names used to save your files. Additionally, ensure that you always specify the criterion device as the last element of the list.
* `criterion`: specify the name of your criterion device, for example: `polar`. Please ensure that this is exactly the same as the name used to save your files.
* `features`: the HRV features that you wish to include in your final group analysis. 

**Please note:** all figures and analyses are functional even if some recordings are missing. For instance, if only 4 out of 10 participants wore the Empatica device, the pipeline will still compute regression, ICC, and Bland-Altman analyses for those participants using the corresponding values from the criterion device. Additionally, all plots employ GUI widgets.

In [ ]:
path = r'***COPY-YOUR-ADDRESS-HERE***/'.replace('\\', '/') # define where the files are located
path = r'C:\Users\msi401\OneDrive - Vrije Universiteit Amsterdam\PhD\Data\Coding\Validation Study\wearable-hrv\tests\test_group/'.replace('\\', '/') # define where the files are located 
conditions = ['sitting', 'arithmetic', 'recovery', 'standing', 'breathing', 'neurotask', 'walking', 'biking']
#devices = ["heartmath", "empatica", "kyto","rhythm", "vu"] ### MAKE SURE TO PUT THE CRITERION DEVICE THE LAST ONE IN THE LIST #####
devices = ["heartmath", "kyto","rhythm", "empatica", "vu"] ### MAKE SURE TO PUT THE CRITERION DEVICE THE LAST ONE IN THE LIST #####

criterion = "vu"
features = ["rmssd", "hf",'pnni_50','mean_hr','sdnn', 'nibi_after_cropping', 'artefact'] # make sure all these features exist, but feel free to add more 

print ("Imported Successfully!")

## Importing data

In [ ]:
data, file_names = wearablehrv.group.import_data (path, conditions, devices, features)

## Handling NaN values

When calculating the individual values, if certain devices or conditions are missing for a participant, the calculation of time or frequency domains may not be performed. As a result, NaN values are replaced for these cases. This interference affects some of the upcoming statistical analyses. By utilizing the `nan_handling` function provided below, any `[nan]` values can be transformed, if present, into empty brackets `[]`.

In [ ]:
data = wearablehrv.group.nan_handling (data, devices, features, conditions)

## Signal Quiality Check and Outliers Exclusion

The main idea is to report signal quiality and to exclude readings from the devices where the signal quality is deemed insufficient according to certain thresholds.

Two key metrics are used to determine signal quality:

- 'nibi_after_cropping': The number of detected beat-to-beat intervals (IBI). A difference of more than `ibi_threshold` (20% by default) between a given device and the criterion device for the same participant and condition indicates a poor signal and the reading is excluded.

- 'artefact': The number of detected artefacts. If the artefacts represent more than `artefact_threshold` (20% by default) of the detected beats in the given device for the same participant and condition, this also indicates a poor signal and the reading is excluded.

If `exclude = True`, all feature values for that participant and condition will be replaced with empty lists. Otherwise, they are kept for further analysis.

Both the `ibi_threshold` and `artefact_threshold` parameters can be tuned according to the specific needs of your analysis. A lower value makes the criteria more stringent, leading to more readings being excluded, while a higher value makes the criteria more lenient, leading to fewer exclusions.

After processing, the code generates two pandas dataframes that can be saved if `save_as_csv = True`:

- `quality_report1.csv`: Detailed report showing for each participant, device, and condition, the number of detected beats and artefacts, and the decision to keep or exclude the data.

- `quality_report2.csv`: Summary report showing the total count and percentage of decisions ("Acceptable", "Poor", "Missing") for each device and condition.

At the end of the process, 'artefact' and 'nibi_after_cropping' data is removed from the `data` and `features` variables.

In [ ]:
data, features = wearablehrv.group.signal_quality (data, path, conditions, devices, features, criterion, file_names, exclude = False, save_as_csv = False, ibi_threshold = 0.40, artefact_threshold = 0.20)

## Saving data

You can save the structured dataset and analyze it as you wish, for example, in another software.

In [ ]:
wearablehrv.group.save_data (data, path, conditions, devices, features, file_names)

## Descriptive plots

In [ ]:
wearablehrv.group.violin_plot (data, conditions, features, devices)

In [ ]:
wearablehrv.group.box_plot (data, conditions, features, devices)

In [ ]:
wearablehrv.group.radar_plot (data, criterion, conditions, features, devices)

In [ ]:
wearablehrv.group.hist_plot (data, conditions, features, devices)

The function `bar_plot` creates bar plots in which the standard error of the mean is represented.

In [ ]:
wearablehrv.group.bar_plot (data, conditions, features, devices)

This function `matrix_plot` creates a heatmap to plot the data. The heatmap displays a specific feature for different participants and devices for a specific condition. The heatmap allows for easy comparison of the feature values across participants and devices. For example, by using this map, it can be observed that a device is systematically overestimating or underestimating a given feature.

**Note:** If you have many participants, you may encounter errors when producing the figure. In this case, you can subselect a portion of your participants by modifying the variable `file_names`. For example, to select the first 20 participants, you can use: `file_names[0:20]`.

In [ ]:
wearablehrv.group.matrix_plot (data, file_names, conditions, features, devices)

The `density_plot` function creates a density plot to compare the distribution of specific feature in different heart rate devices under a given condition.The function uses the seaborn library to create a kernel density estimate plot (kdeplot) for each device, which shows the probability density function of a spesific feature. If the peaks* of the curves are in different locations or have different shapes, this suggests that the spesific feature values may differ across devices. On the other hand, if the curves are similar in shape and location, it suggests that the values are similar across devices.

*The peak in a density plot indicates the point of highest density in the distribution of the data. In other words, it represents the most common or typical value for the variable being plotted.

In [ ]:
wearablehrv.group.density_plot (data, file_names, conditions, features, devices)

## Regression analysis and visualization

The output of this function is a dictionary containing the regression analysis data for each device, feature, and condition. The keys of the dictionary are the names of the devices, and the values are nested dictionaries with keys being the feature names, and the values are another nested dictionary with keys being the condition names and values being the regression analysis data in the form of a dictionary with the following keys: slope, intercept, `r_value`, `p_value`, and `std_err`. All values are calculated against the criterion counterpart. 

In [ ]:
regression_data = wearablehrv.group.regression_analysis (data, criterion, conditions, devices, features, path, save_as_csv=False)

This function generates a scatter plot with a linear regression line for each device and condition combination, comparing the relationship between a specific feature in the data in a given device against the criterion device.

The plot generated by this function has multiple subplots, with each subplot representing a specific device and condition combination. The x-axis of each subplot represents the values of spesific feature in a given device while the y-axis represents the values of the criterion device. Each subplot contains a scatter plot of the data points, with the regression line overlaid on top. The slope and intercept of the regression line are calculated using the regression_data dictionary. The correlation coefficient (r-value) and significance (p-value) of the regression analysis are also calculated using the `regression_data` dictionary and added to the plot as an annotation.

**Limitation of the current function**: This function only calculates the univariate linear regression between inter-beat-intervals of a given device and the criterion. The limitation is that it does not take into account any modulations, such as BMI or skin tone. If you wish to incorporate those factors into your analysis, my suggestion is to use the `save_data` function and run the necessary analyses using the saved data.

In [ ]:
wearablehrv.group.regression_plot (
    regression_data=regression_data, 
    data=data, 
    criterion=criterion, 
    conditions=conditions, 
    devices=devices, 
    features=features,
    width=15, 
    height_per_condition=5, 
    regression_line_style='-', 
    regression_line_color='blue', 
    marker_color='green', 
    font_size=14, 
    show_grid=True, 
    background_color='lightgray'
)

In [ ]:
wearablehrv.group.heatmap_plot (data, criterion, devices, conditions, features)

## Intraclass Correlation Coefficient (ICC)

We use: `pingouin.intraclass_corr (data, targets, raters, ratings)`

- targets: name of column containing the “targets” (the things being rated): participants 
- raters: name of column containing the raters: devices 
- ratings: name of column containing the ratings: values 

The function group_icc_analysis computes Intraclass Correlation Coefficient (ICC) for different heart rate devices, conditions, and HRV features.The ICC ranges from 0 to 1, where 0 indicates no agreement, and 1 indicates perfect agreement. In this function, ICC is computed using the two-way random-effects model, which considers both the inter-rater and intra-rater variability. The function computes ICC separately for each device, feature, and condition combination. The values are calulated for each device against the criterion device. 

The output of the function is a nested dictionary icc_data that stores the ICC values for each combination of device, feature, and condition, where device, feature, and condition are the keys that represent different devices, HRV features, and conditions, respectively. ICC1, ICC2, ICC3, etc., are the keys that represent different ICC metrics computed using different methods (e.g., ICC1 for single-measure ICC or ICC2 for average-measure ICC), and val1, val2, val3, etc., are the corresponding ICC values.

In the case of comparing the feautres for different participants obtained from two devices (a given device against the criterion device), you might be interested in the consistency of the ratings, and therefore ICC1 would be a good choice to interpret.

In [ ]:
icc_data = wearablehrv.group.icc_analysis (data, criterion, devices, conditions, features, path, save_as_csv=False)

In [ ]:
wearablehrv.group.icc_plot (icc_data, conditions, devices, features, font_size=9, cmap="coolwarm")

## Bland-Altman analysis and visualization

Bland-Altman analysis is a statistical method used to compare two quantitative measurement methods.

The first function, `blandaltman_analysis`, performs Bland-Altman analysis for each device against the criterion device, for all features, and conditions. It calculates the bias, standard deviation (SD), and limits of agreement (LOA) for each combination. The results are saved in a nested dictionary structure called blandaltman_data with each combination of device, feature, and condition as the keys.

The second function, `blandaltman_plot` creates a plot for each combination of device and condition agianst the criterion, with each plot showing the Bland-Altman analysis for the selected feature. It shows the difference between the measurements from the two devices on the y-axis and the average of the measurements on the x-axis. The LOA are shown as dashed lines on the plot. The title and labels for each plot are set using the parameters passed to the function.


In [ ]:
blandaltman_data = wearablehrv.group.blandaltman_analysis (data, criterion, devices, conditions, features, path, save_as_csv=False)

In [ ]:
wearablehrv.group.blandaltman_plot (blandaltman_data, data, criterion, conditions, devices, features, 
                         width=10, height_per_plot=5, agreement_bound=1.96, confidenceInterval=95, 
                         percentage=False, mean_diff_color='#FF6347', boundary_color='#20B2AA', pointColour='#8B008B', shade=True)